# Writeup
## Project: Highway Driving / Path Planning

### 1. Compilation
The code compiles correctly. Just execute following in the project folder. 
```
mkdir build && cd build
cmake .. && make
./path_planning
```

### 2. Valid Trajectories 
The car is able to drive at least 4.32 miles without incident, at least if non of the other car's makes a critical driving mistake such as, tough some evasive maneuver has been successful it would certainly violate the jerk limitations: (click on the video below)

[![Crash by NPC](doc/intended_crash.png)](doc/intended_crash.mp4)

The car drives according to the speed limit and the maximum acceleration and jerk are not exceeded most the time. Car does not have collisions, except it can't avoid it because they are caused by the other vehicles. The car stays in its lane, except for the time between changing lanes and thereby is able to change lanes. The lane change algorithm is pretty conservative but relies heavily on constant velocity profiles. As some vehicles randomly decelerate, sometimes issues occur if the vehicle at the target decelerates spontantously.

As everything works normal especially the other vehicle the algorithm is capable driving a very long distance:

![Result](doc/success.png)

### 3. Model Description

The algorithm consists of three essentials parts. The first part is sensor fusion, which predicts and calculates relevant objects for the scenario. These are passed to the second part, the behavior planning. In the behaviorplanning it's validated if a lane change is neccessary and possible given on the sensor fusion data. The target lane is then commanded to the trajectory generation. It's main task is to generate a feasible trajectory by filtering the waypoints and adapting their position and contour to the target lane. Additionally a longituindal trajectory is calculated based on the vehicle predictions.

#### Sensor Fusion

#### Behavior Planning

#### Trajectory Planning

### 4. Reflection


